# Problem 237: Tours on a $4 \times N$ Playing Board

Let $T(n)$ be the number of tours over a $4 \times n$ playing board such that:

<ul><li>The tour starts in the top left corner.</li>
<li>The tour consists of moves that are up, down, left, or right one square.</li>
<li>The tour visits each square exactly once.</li>
<li>The tour ends in the bottom left corner.</li>
</ul>

The diagram shows one tour over a $4 \times 10$ board:

<div class="center">
<img src="./0237.gif?1678992055" class="dark_img" alt=""></div>

$T(10)$ is $2329$. What is $T(10^{12})$ modulo $10^8$?

In [119]:
from dataclasses import dataclass


@dataclass
class Piece:
    left: bool = False
    down: bool = False
    right: bool = False
    up: bool = False
    symbol: str = ""


# Define Unicode box drawing characters
horizontal_line = "\u2500"  # ─
vertical_line = "\u2502"  # │
top_left_corner = "\u250c"  # ┌
top_right_corner = "\u2510"  # ┐
bottom_left_corner = "\u2514"  # └
bottom_right_corner = "\u2518"  # ┘

leftRight = Piece(left=True, right=True, symbol=horizontal_line)
upDown = Piece(up=True, down=True, symbol=vertical_line)
leftDown = Piece(left=True, down=True, symbol=top_right_corner)
leftUp = Piece(left=True, up=True, symbol=bottom_right_corner)
rightUp = Piece(right=True, up=True, symbol=bottom_left_corner)
rightDown = Piece(right=True, down=True, symbol=top_left_corner)

nullPiece = Piece()

pieces = [leftRight, upDown, leftDown, leftUp, rightUp, rightDown]

for piece in pieces:
    print(piece)

Piece(left=True, down=False, right=True, up=False, symbol='─')
Piece(left=False, down=True, right=False, up=True, symbol='│')
Piece(left=True, down=True, right=False, up=False, symbol='┐')
Piece(left=True, down=False, right=False, up=True, symbol='┘')
Piece(left=False, down=False, right=True, up=True, symbol='└')
Piece(left=False, down=True, right=True, up=False, symbol='┌')


In [163]:
from functools import cached_property
from itertools import pairwise
from typing import Tuple


@dataclass
class Column:
    pieces: Tuple[Piece] = tuple()

    @cached_property
    def show(self) -> str:
        return "\n".join(_.symbol for _ in self.pieces)

    @cached_property
    def left_mask(self) -> Tuple[bool]:
        return tuple(_.left for _ in self.pieces)

    @cached_property
    def right_mask(self) -> Tuple[bool]:
        return tuple(_.right for _ in self.pieces)

    @cached_property
    def left_parity(self) -> str:
        if self.pieces[:2] == (leftDown, leftUp):
            return "edge"
        elif self.pieces[2:] == (leftDown, leftUp):
            return "edge"
        elif self.pieces[1:3] == (leftDown, leftUp):
            return "middle"
        else:
            return "none"

    @cached_property
    def right_parity(self) -> str:
        if self.pieces[:2] == (rightDown, rightUp):
            return "edge"
        elif self.pieces[2:] == (rightDown, rightUp):
            return "edge"
        elif self.pieces[1:3] == (rightDown, rightUp):
            return "middle"
        else:
            return "none"

    @cached_property
    def is_valid_column(self) -> bool:
        for p1, p2 in pairwise([nullPiece] + list(self.pieces) + [nullPiece]):
            if p1.down != p2.up:
                return False
        return True

    @cached_property
    def is_valid_middle_column(self) -> bool:
        if not self.is_valid_column:
            return False
        if sum(self.left_mask) not in (2, 4):
            return False
        if sum(self.right_mask) not in (2, 4):
            return False
        if self.left_mask == (True, True, False, False) and self.right_mask == (
            False,
            False,
            True,
            True,
        ):
            return False
        if self.left_mask == (False, False, True, True) and self.right_mask == (
            True,
            True,
            False,
            False,
        ):
            return False
        return True

    @cached_property
    def is_valid_start_column(self) -> bool:
        if not self.is_valid_column:
            return False
        if not self.is_valid_middle_column:
            return False
        if not self.left_mask == (True, False, False, True):
            return False
        return True

    @cached_property
    def is_valid_end_column(self) -> bool:
        if not self.is_valid_column:
            return False
        if sum(self.right_mask) > 0:
            return False
        if sum(self.left_mask) not in [2, 4]:
            return False
        return True


# sanity checks
print(Column([upDown, leftRight, leftRight, leftRight]).is_valid_column)  # False
print(Column([leftRight, leftRight, leftRight, leftRight]).is_valid_column)  # True
print(Column([leftRight, leftDown, rightUp, leftRight]).is_valid_column)  # True
print(Column([leftRight, leftDown, rightUp, leftUp]).is_valid_column)  # False


print(
    Column([leftRight, leftRight, leftRight, leftRight]).is_valid_middle_column
)  # True
print(
    Column([leftRight, leftRight, leftRight, leftRight]).is_valid_start_column
)  # False
print(Column([leftRight, leftRight, leftRight, leftRight]).is_valid_end_column)  # False
print(Column([leftRight, rightDown, rightUp, leftRight]).is_valid_start_column)  # True
print(Column([leftDown, upDown, upDown, leftUp]).is_valid_end_column)  # True

False
True
True
False
True
False
False
True
True


In [162]:
from itertools import product

start_columns = []
mid_columns = []
end_columns = []

for _columns in product(pieces, repeat=4):
    column = Column(tuple(_columns))
    if column.is_valid_start_column:
        start_columns.append(column)
    if column.is_valid_middle_column:
        mid_columns.append(column)
    if column.is_valid_end_column:
        end_columns.append(column)

print("Number of valid start columns:", len(start_columns))
print("Number of valid middle columns:", len(mid_columns))
print("Number of valid end columns:", len(end_columns))
print("")
for i, s in enumerate(start_columns):
    print(f"Column {i}")
    print(s.show)
    print("")

Number of valid start columns: 4
Number of valid middle columns: 15
Number of valid end columns: 2

Column 0
─
┌
│
┘

Column 1
─
┌
└
─

Column 2
┐
│
└
─

Column 3
┐
└
┌
┘



In [199]:
from typing import List, Type


@dataclass
class PartialBoard:
    left_mask: Tuple[bool]
    right_mask: Tuple[bool]
    left_parity: str
    right_parity: str
    length: int
    degeneracy: int
    cache: List[List[Column]] = None

    @classmethod
    def from_column(cls, column: Column, cache=False) -> Type["PartialBoard"]:
        return cls(
            left_mask=column.left_mask,
            right_mask=column.right_mask,
            left_parity=column.left_parity,
            right_parity=column.right_parity,
            length=1,
            degeneracy=1,
            cache=[[column]] if cache else None,
        )

    def add(self, other: Type["PartialBoard"], mod=10**8) -> Type["PartialBoard"]:
        if self.right_mask != other.left_mask:
            return None
        if self.right_parity == other.left_parity and self.right_parity != "none":
            return None
        left_mask = self.left_mask
        right_mask = other.right_mask
        left_parity = (
            self.left_parity
            if (self.left_parity, self.right_parity) != ("none", "none")
            else other.left_parity
        )
        right_parity = (
            other.right_parity
            if (other.left_parity, other.right_parity) != ("none", "none")
            else self.right_parity
        )

        cache = (
            None
            if self.cache is None and other.cache is None
            else [l + r for l in self.cache for r in other.cache]
        )

        return PartialBoard(
            left_mask=left_mask,
            right_mask=right_mask,
            left_parity=left_parity,
            right_parity=right_parity,
            length=self.length + other.length,
            degeneracy=(self.degeneracy % mod) * (other.degeneracy % mod) % mod,
            cache=cache,
        )

    def image(self, columns: List[Column]) -> str:
        string = [""] * 4
        for col in columns:
            for i in range(4):
                string[i] += col.pieces[i].symbol
        return "\n".join(string)


def combine_partial_boards(
    left: List[PartialBoard], right: List[PartialBoard], mod=10**8
) -> List[PartialBoard]:
    partial_boards = dict()
    for _left, _right in product(left, right):
        combined = _left.add(_right, mod)
        if combined is not None:
            key = (
                combined.left_mask,
                combined.right_mask,
                combined.left_parity,
                combined.right_parity,
            )
            if key not in partial_boards:
                partial_boards[key] = combined
            else:
                partial_boards[key].degeneracy += combined.degeneracy
                partial_boards[key].degeneracy %= mod
                if combined.cache is not None:
                    partial_boards[key].cache += combined.cache
    return list(partial_boards.values())

In [197]:
from itertools import count


def T(n: int, show=False, mod=10**8) -> int:

    starts = [PartialBoard.from_column(col, cache=show) for col in start_columns]
    mids = [PartialBoard.from_column(col, cache=show) for col in mid_columns]
    ends = [PartialBoard.from_column(col, cache=show) for col in end_columns]

    # partition n-2 into powers of 2
    powers_of_two = []
    _n = n - 2
    for exponent in count(0):
        _n, rem = divmod(_n, 2)
        if rem == 1:
            powers_of_two.append(exponent)
        if _n == 0:
            break

    partial_boards = {0: mids}
    for exp in range(1, exponent + 1):
        partial_boards[exp] = combine_partial_boards(
            partial_boards[exp - 1], partial_boards[exp - 1]
        )

    result = starts.copy()
    for exp in powers_of_two:
        result = combine_partial_boards(result, partial_boards[exp])
    result = combine_partial_boards(result, ends)[0]
    if show:
        for board in result.cache:
            print("")
            print(result.image(board))

    return result.degeneracy % 10**8


print("T(5) =", T(5, show=True))


────┐
┌──┐│
│┌─┘│
┘└──┘

┐┌──┐
│└─┐│
└──┘│
────┘

────┐
┌┐┌┐│
│││││
┘└┘└┘

──┐┌┐
┌┐│││
││└┘│
┘└──┘

──┐┌┐
┌┐└┘│
││┌┐│
┘└┘└┘

┐┌──┐
││┌┐│
└┘│││
──┘└┘

┐┌┐┌┐
│││││
└┘└┘│
────┘

┐┌┐┌┐
││└┘│
└┘┌┐│
──┘└┘

┐┌──┐
└┘┌┐│
┌┐│││
┘└┘└┘

┐┌┐┌┐
└┘│││
┌┐└┘│
┘└──┘

┐┌┐┌┐
└┘└┘│
┌┐┌┐│
┘└┘└┘

────┐
┌──┐│
└─┐││
──┘└┘

──┐┌┐
┌─┘││
└──┘│
────┘

────┐
┌──┐│
└┐┌┘│
─┘└─┘

─┐┌─┐
┌┘└┐│
└──┘│
────┘

────┐
┌─┐┌┘
│┌┘└┐
┘└──┘

┐┌──┐
│└┐┌┘
└─┘└┐
────┘

────┐
┌┐┌─┘
││└─┐
┘└──┘

┐┌──┐
││┌─┘
└┘└─┐
────┘

┐┌──┐
└┘┌─┘
┌┐└─┐
┘└──┘

────┐
┌───┘
└───┐
────┘

────┐
┌─┐┌┘
└┐│└┐
─┘└─┘

─┐┌─┐
┌┘│┌┘
└─┘└┐
────┘
T(5) = 23


In [198]:
print("T(5) =", T(5))
print("T(10) =", T(10))
print("T(20) =", T(20))
print("T(100) =", T(100))
print("T(1000) =", T(1000))
print("T(100000) =", T(100000))
print("T(10**12) =", T(10**12))

T(5) = 23
T(10) = 2329
T(20) = 25924088
T(100) = 65378344
T(1000) = 64570120
T(100000) = 52387936
T(10**12) = 15836928
